## notes about standard errors, confidence intervals and hypothesis tests

recall the setup of our AB test:

we have a treatment/intervention $ X \in \{0,1\}$ and we have an outcome variable $Y$

it's then very easy to calculate the average of $Y$ for the treatment and control groups and take the difference:

`avg(Y | X = 1) - avg(Y | X = 0)`

this is a measure of association between $X$ and $Y$. If the difference is large and positive -- that is very suggestive and we will WANT to believe that the treatment is the reason why. But when we decompose this simple difference as so:

`avg(Y | X = 1) - avg(Y | X = 0) = avg treatment effect (on the treated) + bias`

we realize we need to be mindful that our results are not due to a systematic difference between treatment and control groups that is unrelated to the treatment.

as we saw, the nice thing about AB tests is that they are designed to eliminate bias through random assignment of $X$

if bias is zero in an AB test, then `avg(Y | X = 1) - avg(Y | X = 0)` is the treatment effect. 

this brings us to a **question**:

if `avg(Y | X = 1) - avg(Y | X = 0)` is the unbiased treatment effect then why did we bother to do a hypothesis test?

## bias vs noise

randomization helps remove bias, but findings can still be misleading due to pure chance, which is called noise.

**bias**: assigning people to treatment/control based on their lifestyle. minors with lots of free time go to the treatment group, and busy working adults to the control group. the treatment group plays more — but they were always going to play more!

**noise**: andy loves cookie cats and schedules gaming sessions for thursdays, but gets sick and can’t play. beatrice rarely plays but starts dating someone who loves cookie cats and plays more because of it.

imagine this randomness for all ~90k participants in the AB test. this is noise.

putting this in terms of our favorite equation:

`avg(Y | X = 1) - avg(Y | X = 0) = avg treatment effect on the treated + bias`

`avg(Y | X = 1) - avg(Y | X = 0) = real avg treatment effect on the treated + noise + bias`

the first equation shows the observed difference includes bias. the second equation shows the observed difference includes the real treatment effect, noise, and bias.


## essential justification for hypothesis testing

we talked about this before, recall:

assume we've used random assignment to push bias down to ~0. and now we want to explain where the variability in the data comes from -- there's really only three options:

**option 1**: all the variability is due to real, systematic effect of the treatment

**option 2**: all the variability is due to randomness (noise) 

**option 3:** the variability is some combination of real effect and noise

it is extremely unlikely **option 1** is true

~~all the variability is due to real, systematic effect of the treatment~~

> so we are left to decide which we believe more: **option 2** or **option 3**

when we use a hypothesis test, what we are "testing" is how much we should believe **option 2**

a very small p-value makes us disinclined to believe it, and more inclined to favor **option 3**